In [1]:
%load_ext autoreload
%autoreload 2
import sys
sys.path.append('..')

## Download some sample data

In [173]:
import priceanalytics.data as data

big_df = data.alpaca_download_single(
    'AAPL',     
    past_days=30, 
    interval=5  
)
dfs = data.split_alpaca_on_day(big_df)

In [189]:
s = """
$$ \text{Opening and closing price series are defined by}\  O(t) \ \text{and} \ C(t) \ \text{respectively, while}\ \alpha\ \text{is an arbitrary constant.} $$

$$ u(t) = \max( O(t),\ C(t),\ u(t-1) - \frac{u(t-1) - C(t)}{\alpha}) $$

$$ d(t) = \min( O(t),\ C(t),\ d(t-1) - \frac{C(t) - d(t-1)}{\alpha}) $$

$$ U(t) = \max( O(t)^2,\ C(t)^2,\ U(t-1)^2 - \frac{U(t-1) - C(t)^2}{\alpha}) $$

$$ D(t) = \min( O(t)^2,\ C(t)^2,\ D(t-1)^2 - \frac{C(t)^2 - D(t-1)}{\alpha}) $$

$$ B(t) = \sqrt{U(t)-u(t)^2} $$

$$ b(t) = -\sqrt{D(t)-d(t)^2} $$

$$ \text{Enter a long position when }\ \nabla B(t) < 0\ \text{and enter a short position when }\ \nabla b(t) > 0 \text{.} $$
"""

s=s.replace('\t', '\\t')
s=s.replace('\na', '\\na')
s=s.replace('\al', '\\al')
s=s.replace('\fr', '\\fr')

if '(t)' in s:
    s=s.replace('(t)', '_t')
    s=s.replace('(t-1)', '_{t-1}')
else:
    s=s.replace('_t', '(t)')
    s=s.replace('_{t-1}', '(t-1)')

print(s)


$$ \text{Opening and closing price series are defined by}\  O_t \ \text{and} \ C_t \ \text{respectively, while}\ \alpha\ \text{is an arbitrary constant.} $$

$$ u_t = \max( O_t,\ C_t,\ u_{t-1} - \frac{u_{t-1} - C_t}{\alpha}) $$

$$ d_t = \min( O_t,\ C_t,\ d_{t-1} - \frac{C_t - d_{t-1}}{\alpha}) $$

$$ U_t = \max( O_t^2,\ C_t^2,\ U_{t-1}^2 - \frac{U_{t-1} - C_t^2}{\alpha}) $$

$$ D_t = \min( O_t^2,\ C_t^2,\ D_{t-1}^2 - \frac{C_t^2 - D_{t-1}}{\alpha}) $$

$$ B_t = \sqrt{U_t-u_t^2} $$

$$ b_t = -\sqrt{D_t-d_t^2} $$

$$ \text{Enter a long position when }\ \nabla B_t < 0\ \text{and enter a short position when }\ \nabla b_t > 0 \text{.} $$



## Square Bounds algorithm

<!--
$$ \text{Opening and closing price series are defined by}\  O_t \ \text{and} \ C_t \ \text{respectively, while}\ \alpha\ \text{is an arbitrary constant.} $$

$$ u_t = \max( O_t,\ C_t,\ u_{t-1} - \frac{u_{t-1} - C_t}{\alpha}) $$

$$ d_t = \min( O_t,\ C_t,\ d_{t-1} - \frac{C_t - d_{t-1}}{\alpha}) $$

$$ U_t = \max( O_t^2,\ C_t^2,\ U_{t-1}^2 - \frac{U_{t-1} - C_t^2}{\alpha}) $$

$$ D_t = \min( O_t^2,\ C_t^2,\ D_{t-1}^2 - \frac{C_t^2 - D_{t-1}}{\alpha}) $$

$$ B_t = \sqrt{U_t-u_t^2} $$

$$ b_t = -\sqrt{D_t-d_t^2} $$

$$ \text{Enter a long position when }\ \nabla B_t < 0\ \text{and enter a short position when }\ \nabla b_t > 0 \text{.} $$
-->


$$ \text{Opening and closing price series are defined by}\  O(t) \ \text{and} \ C(t) \ \text{respectively, while}\ \alpha\ \text{is an arbitrary constant.} $$

$$ u(t) = \max( O(t),\ C(t),\ u(t-1) - \frac{u(t-1) - C(t)}{\alpha}) $$

$$ d(t) = \min( O(t),\ C(t),\ d(t-1) - \frac{C(t) - d(t-1)}{\alpha}) $$

$$ U(t) = \max( O^2(t),\ C(t)^2,\ U^2(t-1) - \frac{U(t-1) - C^2(t)}{\alpha}) $$

$$ D(t) = \min( O^2(t),\ C(t)^2,\ D^2(t-1) - \frac{C^2(t) - D(t-1)}{\alpha}) $$

$$ B(t) = \sqrt{U(t)-u^2(t)} $$

$$ b(t) = -\sqrt{D(t)-d^2(t)} $$

$$ \text{Enter a long position when }\ \ \frac{dB}{dt}(t) < 0\ \text{and enter a short position when }\ \ \frac{db}{dt}(t) > 0 \text{.} $$


In [ ]:
import pandas as pd
import numpy as np
import priceanalytics.plot as plot
import matplotlib.pyplot as plt

def roll(a, window):
    shape = a.shape[:-1] + (a.shape[-1] - window + 1, window)
    strides = a.strides + (a.strides[-1],)
    return np.lib.stride_tricks.as_strided(a, shape=shape, strides=strides)

def sb(close, _open):

    N = len(close)
#     alpha = 2 / (_len + 1)
#     alpha_sig = 2 / (sig_len + 1)
    alpha = 1/10

    up1, up2, dn1, dn2, bull, bear, sig = np.zeros((7, N))

    up1[0] = dn1[0] = close[0]
    up2[0] = dn2[0] = close[0] ** 2

    #print(alpha, alpha_sig, up1[0], dn1[0], up2[0], dn2[0], sig[0])
       
    for i in range(1, N):
        
        up1[i] = max(close[i], _open[i], up1[i - 1] - alpha * (up1[i - 1] - close[i]))
        dn1[i] = min(close[i], _open[i], dn1[i - 1] + alpha * (close[i] - dn1[i - 1]))

        up2[i] = max(close[i] ** 2, _open[i] ** 2, up2[i - 1] - alpha * (up2[i - 1] - close[i] ** 2))
        dn2[i] = min(close[i] ** 2, _open[i] ** 2, dn2[i - 1] + alpha * (close[i] ** 2 - dn2[i - 1]))
     
        #print(up1[i], np.sqrt(up2[i]), dn1[i], np.sqrt(dn2[i]))
        
        
        bull[i] = np.sqrt(up2[i] - up1[i] ** 2)
        bear[i] = np.sqrt(dn2[i] - dn1[i] ** 2)

        #sig[i] = sig[i - 1] + alpha_sig * (np.maximum(bull[i], bear[i]) - sig[i - 1])
        
    bear *= -1
        
    bull_1st_ddt = np.gradient(bull)
    bull_2nd_ddt = np.gradient(bull_1st_ddt)
    bull_3rd_ddt = np.gradient(bull_2nd_ddt)

    bear_1st_ddt = np.gradient(bear)
    bear_2nd_ddt = np.gradient(bear_1st_ddt)
    bear_3rd_ddt = np.gradient(bear_2nd_ddt)
    
    bull_ma = pd.Series(bull).rolling(3).mean().to_numpy()
    bear_ma = pd.Series(bear).rolling(3).mean().to_numpy()
        
    inpos = False
    longs, shorts = [], []
    for i in range(1, N-1):
        if bull_2nd_ddt[i] < 0 and not inpos:
            inpos = True
            longs.append(i)
        elif bear_2nd_ddt[i] > 0 and inpos:
            inpos = False
            shorts.append(i)
            
    with plot.MultiPlot(2, size=(18, 9 * 3)) as m:
        plt.plot(close)
        for long in longs:
            plt.axvline(x=long, color="#00FF00")
        for short in shorts:
            plt.axvline(x=short, color="#FF0000")
        
        #plt.plot(up1)
        #plt.plot(np.sqrt(up2))
            
        #plt.plot(dn1)
        #plt.plot(np.sqrt(dn2))
     
        m.next()
        
        plt.plot(bull)
#         plt.plot(bull_ma)

        plt.plot(bear)    
    
#         plt.plot(bear_ma)

        
#         m.next()
        
#         plt.plot(bull_1st_ddt)
#         plt.plot(bull_2nd_ddt)
#         #bear_1st_ddt, bear_2nd_ddt
        
      
    
    #return df


def test_sb(df):
    
    
    close = df['close'].to_numpy()
    _open = df['open'].to_numpy()
    
    sb(close, _open)

test_sb(dfs[-12].copy())

## Todo tommorow

- play around w/ the formula a little more
- maybe mess with alpha a bit (; 
- implement a proper backtesting notebook
- maybe make a testing script for tuning alpha parameters + many datapoints
- implement gradient algorithm in go (cry) with testcases (supercry)